In [132]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn import preprocessing
from sklearn import tree
from typing import List
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

In [133]:

import random
import math
def generateData(n):
    data = []
    for i in range(0,n):
        minggu = random.randint(1, 392)
        totaldonor = 1 if math.floor(minggu/8) == 0 else random.randint(1,math.floor(minggu/8))
        data.append([minggu, totaldonor,0.0 if totaldonor == 0 else 1.0 if minggu/totaldonor <= 12.0 else 0.0])
    return data

data = generateData(300)

real_df = pd.DataFrame(data,columns=["Jumlah Minggu", "Total donor per tahun","aktif"])
real_df.head()

,Jumlah Minggu,Total donor per tahun,aktif
0,307,5,0.0
1,304,5,0.0
2,106,13,1.0
3,299,22,0.0
4,210,26,1.0


In [134]:
scalerX = MinMaxScaler()


def preprocessing(df:pd.DataFrame,categorical_columns:List[str],scaling:bool, unused_columns:List[str], label:str):
  df.dropna() # drop row bila salah 1 atau lebih ada NaN
  # df.dropna(subset=[1]) # drop row bila column 1 NaN

  if len(categorical_columns) > 0:
    df = pd.get_dummies(data=df,columns=categorical_columns)
  # for column in df.columns:
  #   le = LabelEncoder()
  #   le.fit(df[column])
  #   df[column] = le.transform(df[column])
  
  # harus displit dulu antara features dan label, karena bila di minmax scalling dulu
  # nama kolom akan hilang, berubah menjadi array 2d
  unused_columns.append(label) # hapus kolom data column dan juga label
  X = df.drop(columns=unused_columns) 
  Y = df[label]
  Y = np.array(Y).reshape((-1,1))
  if scaling:
    global scalerX
    scalerX.fit(X.to_numpy())
    print("X")
    print(X)
    X = scalerX.transform(X.to_numpy())

    scaler = MinMaxScaler()
    scaler.fit(Y)
    Y = scaler.transform(Y)

  # always produce the same result (only random once)
  # x_train, x_test = train_test_split(X, test_size=test_size, random_state=25)
  # y_train, y_test = train_test_split(Y, test_size=test_size, random_state=25)

  # x_train = np.array(x_train)
  # y_train = np.array(y_train).reshape((-1,1))
  # x_test  = np.array(x_test)
  # y_test  = np.array(y_test).reshape((-1,1))
  X = np.array(X)
  Y = np.array(Y).reshape((-1,1))
  
  return X, Y


In [135]:
real_df.head()

,Jumlah Minggu,Total donor per tahun,aktif
0,307,5,0.0
1,304,5,0.0
2,106,13,1.0
3,299,22,0.0
4,210,26,1.0


In [136]:
x_train, y_train = preprocessing(real_df,
                                                 scaling=False,
                                                 categorical_columns=[],
                                                  unused_columns=[],
                                                 label="aktif")

In [137]:
from sklearn import tree
model = tree.DecisionTreeClassifier()
model.fit(x_train, y_train)

DecisionTreeClassifier()

In [138]:
scaled_x_train, scaled_y_train = preprocessing(real_df,
                                                 scaling=True,
                                                 categorical_columns=[],
                                                 unused_columns=[],
                                                 label="aktif")

X
     Jumlah Minggu  Total donor per tahun
0              307                      5
1              304                      5
2              106                     13
3              299                     22
4              210                     26
..             ...                    ...
295            232                     18
296            163                     19
297            349                     16
298            379                     12
299            143                      9

[300 rows x 2 columns]


In [139]:
tranfusion_df = pd.read_csv("transfusion.csv")
tranfusion_df = tranfusion_df.drop(labels=["Monetary (c.c. blood)","Recency (months)","whether he/she donated blood in March 2007"],axis=1)
tranfusion_df = tranfusion_df[["Time (months)","Frequency (times)"]]

for i, row in tranfusion_df.iterrows():
    tranfusion_df.at[i,"Time (months)"] = row["Time (months)"]*4

tranfusion_df.rename(columns = {'Time (weeks)':'Time (weeks)'}, inplace = True)
tranfusion_arr = tranfusion_df.to_numpy()

# tranfusion_arr =np.array(list(map(toWeek,tranfusion_arr)))
scaled_tranfusion_arr = scalerX.transform(tranfusion_arr)

In [140]:
scaled_y_train

array([[0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],

In [141]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(activation="relu",solver="lbfgs",shuffle=True)
mlp.fit(scaled_x_train, scaled_y_train.ravel())

MLPClassifier(solver='lbfgs')

expectValue digunakan untuk menvalidasi output dari model MLP dan Decision Tree  
Berdasarkan https://www.halodoc.com/artikel/berapa-kali-boleh-donor-darah-dalam-setahun dikatakan aktif(pria) bila interval waktu donor 8-14 minggu

In [142]:
expectValue = list(map(lambda x: 1.0 if x[0]/x[1] <= 12.0 else 0.0,tranfusion_arr))

 **source dataset  : https://www.kaggle.com/datasets/ninalabiba/blood-transfusion-dataset?resource=download**

In [143]:
DTresult = model.predict(tranfusion_arr) 
MLPresult = mlp.predict(scaled_tranfusion_arr)

tranfusion_df["Predicted Value (MLP)"] = MLPresult
tranfusion_df["Predicted Value (DT)"] = DTresult
tranfusion_df["Expected Value"] = expectValue

In [144]:
from IPython.display import display, HTML
# tranfusion_df
display(HTML("<h2>Hasil Perbandingan Deep Learning (MLP) dan Decision Tree (DT)</h2><p>source dataset: <a href='https://www.kaggle.com/datasets/ninalabiba/blood-transfusion-dataset?resource=download'>kaggle</a></p>"))
display(HTML(f"<div style='overflow-y:auto;max-height:200px'>{tranfusion_df.to_html()}</div>"))

,Time (months),Frequency (times),Predicted Value (MLP),Predicted Value (DT),Expected Value
0,392,50,1.0,1.0,1.0
1,112,13,1.0,1.0,1.0
2,140,16,1.0,1.0,1.0
3,180,20,1.0,1.0,1.0
4,308,24,0.0,0.0,0.0
5,16,4,1.0,1.0,1.0
6,56,7,1.0,1.0,1.0
7,140,12,1.0,1.0,1.0
8,88,9,1.0,1.0,1.0
9,392,46,1.0,1.0,1.0


In [145]:
# Modified input
def modified(data,no_label=False):
    modified_data = []
    for i, d in enumerate(data):
        if no_label:
            modified_data.append([d[0]/d[1]]) # kedua input digabung
        else:
            modified_data.append([d[0]/d[1],d[2]]) # kedua input digabung
    return modified_data
modified_data = modified(data)
modified_data
modified_df = pd.DataFrame(modified_data,columns=["Jumlah Minggu","aktif"])
modified_df.head()

,Jumlah Minggu,aktif
0,61.400000,0.0
1,60.800000,0.0
2,8.153846,1.0
3,13.590909,0.0
4,8.076923,1.0


In [146]:
modifiedDT = tree.DecisionTreeClassifier()
modified_x_train, modified_y_train = preprocessing(modified_df, scaling=False,
                                                 categorical_columns=[],
                                                  unused_columns=[],
                                                 label="aktif")
modifiedDT.fit(modified_x_train, modified_y_train)
tranfusion_arr_modified = modified(tranfusion_arr,no_label=True)
modifiedDTresult = modifiedDT.predict(tranfusion_arr_modified) 

In [147]:
tranfusion_df["Predicted Value (modified DT)"] = modifiedDTresult

display(HTML("<h2>Hasil Perbandingan Deep Learning (MLP) dan Decision Tree (DT) dan Modified Decision Tree (DT)</h2><p>source dataset: <a href='https://www.kaggle.com/datasets/ninalabiba/blood-transfusion-dataset?resource=download'>kaggle</a></p>"))
display(HTML(f"<div style='overflow-y:auto;max-height:200px'>{tranfusion_df.to_html()}</div>"))

,Time (months),Frequency (times),Predicted Value (MLP),Predicted Value (DT),Expected Value,Predicted Value (modified DT)
0,392,50,1.0,1.0,1.0,1.0
1,112,13,1.0,1.0,1.0,1.0
2,140,16,1.0,1.0,1.0,1.0
3,180,20,1.0,1.0,1.0,1.0
4,308,24,0.0,0.0,0.0,0.0
5,16,4,1.0,1.0,1.0,1.0
6,56,7,1.0,1.0,1.0,1.0
7,140,12,1.0,1.0,1.0,1.0
8,88,9,1.0,1.0,1.0,1.0
9,392,46,1.0,1.0,1.0,1.0


In [148]:
# fig = plt.figure(figsize=(25,20))
# _ = tree.plot_tree(model, 
#                    feature_names=["Donor 1 tahun terakhir","Jumlah hari terakhir donor", "frekuensi donor per tahun"],  
#                    class_names=["aktif"],
#                    filled=True)

import graphviz
# DOT data
dot_data = tree.export_graphviz(modifiedDT, out_file=None, 
                                feature_names=["rata-rata interval donor"],  
                                class_names=["Aktif","non Aktif"],
                                filled=True)

# Draw graph
graph = graphviz.Source(dot_data, format="png") 
graph.render("decision_tree_graphivz")

'decision_tree_graphivz.png'

In [149]:
import pickle
pickle.dump(modifiedDT, open("model", 'wb'))